In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  
from selenium.webdriver.common.by import By
import time 
from seleniumwire import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver


# Prepare zip codes data

In [205]:
zip_codes = pd.read_csv('DECENNIALDHC2020.P1-Data.csv')[1:].drop(columns=['Unnamed: 3',
                                                                         'NAME']).reset_index(drop=True)
zip_codes.columns = ['zip', 'population']
zip_codes['zip'] = zip_codes['zip'].apply(lambda x: x[9:])

In [206]:
zip_codes

,zip,population
0,00601,17242
1,00602,37548
2,00603,49804
3,00606,5009
4,00610,25731
...,...,...
33769,99923,66
33770,99925,784
33771,99926,1562
33772,99927,49


In [187]:
cities = pd.read_csv('Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
cities = cities[['RegionName', 'State', 'City']]
cities.columns = ['zip', 'State', 'City']
cities['zip'] = cities['zip'].astype(str)
cities['zip'] = cities['zip'].apply(lambda x: x.zfill(5))

In [188]:
cities

,zip,State,City
0,77494,TX,Katy
1,08701,NJ,Lakewood
2,77449,TX,Katy
3,11368,NY,New York
4,77084,TX,Houston
...,...,...,...
26363,14441,NY,Dresden
26364,16682,PA,Sproul
26365,46799,IN,Zanesville
26366,22731,VA,Aroda


In [207]:
zip_codes = zip_codes.merge(cities, on='zip', how='left').dropna(subset='City').reset_index(drop=True)
zip_codes['City'] = zip_codes['City'].apply(lambda x: x.lower())
zip_codes

,zip,population,State,City
0,01001,16984,MA,agawam
1,01002,27558,MA,amherst
2,01005,4900,MA,barre
3,01007,15423,MA,belchertown
4,01008,1317,MA,blandford
...,...,...,...,...
25266,99801,30088,AK,juneau
25267,99824,2111,AK,juneau
25268,99835,8446,AK,sitka
25269,99901,13942,AK,ketchikan


In [208]:
top_cities = pd.read_excel('Cities Seed List.xlsx')
top_cities['City'] = top_cities['City'].apply(lambda x: x.lower())
top_cities.drop_duplicates(subset= ['City', 'State'], keep='first', inplace=True)
top_cities

,City,State
0,new york,New York
1,los angeles,California
2,chicago,Illinois
3,houston,Texas
4,phoenix,Arizona
...,...,...
574,bennington,Vermont
575,arlington,Virginia
577,wheeling,West Virginia
578,racine,Wisconsin


In [209]:
top_cities_set = set(list(top_cities['City']))

In [210]:
zip_codes = zip_codes[zip_codes['City'].isin(top_cities_set)].reset_index(drop=True)

In [211]:
zip_codes.columns = ['zip', 'population', 'Postal', 'City']

In [212]:
zip_codes = zip_codes.merge(abbreviations, on='Postal', how='left')

In [213]:
zip_codes.drop(columns=['Postal'], inplace=True)

In [214]:
zip_codes

,zip,population,City,State
0,01050,2467,huntington,Massachusetts
1,01083,2549,warren,Massachusetts
2,01092,1343,warren,Massachusetts
3,01103,2545,springfield,Massachusetts
4,01104,23598,springfield,Massachusetts
...,...,...,...,...
4907,99709,28613,fairbanks,Alaska
4908,99712,13309,fairbanks,Alaska
4909,99801,30088,juneau,Alaska
4910,99824,2111,juneau,Alaska


In [215]:
# Merge the two DataFrames on 'City' and 'State'
merged_df = pd.merge(top_cities, zip_codes, on=['City', 'State'], how='left')

**South Fulton** - is in the Atlanta metropolitan area

**East Honolulu** consists of the area directly east of the center of Honolulu

**Paradise** is an unincorporated town and census-designated place (CDP) in Clark County, Nevada, United States, adjacent to the city of Las Vegas

In [216]:
merged_df[(pd.isna(merged_df.population))]

,City,State,zip,population
3147,south fulton,Georgia,NaN,NaN
3598,paradise,Nevada,NaN,NaN


In [217]:
merged_df.dropna(subset=['population'], inplace=True)

In [220]:
# Find the row with the maximum population for each combination of 'City' and 'State'
top_zip_codes = merged_df.loc[merged_df.groupby(['City', 'State'])['population'].idxmax()]

In [221]:
top_zip_codes

,City,State,zip,population
3472,aberdeen,South Dakota,57401,32275
2899,abilene,Texas,79605,30239
2443,akron,Ohio,44303,7031
3329,albany,New York,12203,28210
1255,albuquerque,New Mexico,87121,77802
...,...,...,...,...
2985,wilmington,North Carolina,28412,42758
2068,winston-salem,North Carolina,27107,49834
3281,woodbridge,New Jersey,07095,19839
2271,worcester,Massachusetts,01607,9378


In [232]:
scraped = pd.read_excel('all_zip_codes.xlsx')
scraped.drop(columns=['population', 'State'], inplace=True)
scraped

,zip,City,Brake Pad Replacement,Water Pump Replacement,Spark Plug Replacement,Alternator Replacement,Starter Replacement,Door Mirror Replacement,Battery Replacement,AC Recharge and Leak Inspection,"7,500 mile service",Tire Balancing
0,75949,huntington,$303 to $406,$779 to $942,$249 to $269,$838 to $879,$725 to $763,$955 to $966,$332 to $341,$295 to $350,$89 to $93,$25 to $31
1,71671,warren,$303 to $406,$779 to $942,$249 to $269,$838 to $879,$725 to $763,$955 to $966,$332 to $341,$295 to $350,$89 to $93,$25 to $31
2,1151,springfield,$358 to $476,"$952 to $1,162",$276 to $304,$894 to $950,$777 to $830,$970 to $986,$344 to $356,$369 to $444,$94 to $99,$34 to $43
3,1254,richmond,$317 to $424,$825 to $999,$256 to $278,$853 to $898,$739 to $781,$959 to $971,$335 to $345,$315 to $374,$90 to $94,$27 to $34
4,1378,warwick,$339 to $452,"$893 to $1,086",$267 to $292,$875 to $925,$760 to $807,$965 to $979,$340 to $351,$344 to $412,$92 to $97,$31 to $39
...,...,...,...,...,...,...,...,...,...,...,...,...
253,98682,vancouver,$486 to $587,"$1,221 to $1,505",$320 to $360,"$980 to $1,060",$859 to $935,"$994 to $1,017",$363 to $381,$485 to $591,$102 to $110,$48 to $62
254,99204,spokane,$354 to $471,"$938 to $1,144",$274 to $302,$889 to $944,$773 to $825,$969 to $984,$343 to $355,$363 to $436,$94 to $99,$33 to $42
255,99518,anchorage,$338 to $450,"$888 to $1,081",$266 to $291,$873 to $924,$758 to $806,$965 to $979,$340 to $351,$342 to $409,$92 to $97,$30 to $39
256,99654,wasilla,$325 to $434,"$847 to $1,028",$260 to $283,$860 to $907,$860 to $907,$961 to $974,$337 to $347,$325 to $387,$91 to $95,$28 to $36


In [233]:
scraped['zip'] = scraped['zip'].astype(str)
scraped['zip'] = scraped['zip'].apply(lambda x: x.zfill(5))

In [240]:
to_scrape = pd.merge(top_zip_codes, scraped, on=['City', 'zip'], how='left')
to_scrape = to_scrape.sort_values(by='Brake Pad Replacement', ascending=True).reset_index(drop=True)
#to_scrape.to_excel('to_scrape.xlsx', index=False)
to_scrape

,City,State,zip,population,Brake Pad Replacement,Water Pump Replacement,Spark Plug Replacement,Alternator Replacement,Starter Replacement,Door Mirror Replacement,Battery Replacement,AC Recharge and Leak Inspection,"7,500 mile service",Tire Balancing
0,missoula,Montana,59804,8193,$301 to $404,$775 to $936,$248 to $268,$837 to $877,$724 to $762,$955 to $966,$331 to $340,$293 to $347,$89 to $93,$24 to $31
1,cheyenne,Wyoming,82001,37528,$301 to $404,$775 to $936,$248 to $268,$837 to $877,$724 to $762,$955 to $966,$331 to $340,$293 to $347,$89 to $93,$24 to $31
2,charleston,West Virginia,25311,9487,$301 to $404,$775 to $936,$248 to $268,$837 to $877,$724 to $762,$955 to $966,$331 to $340,$293 to $347,$89 to $93,$24 to $31
3,south burlington,Vermont,05403,20166,$301 to $404,$775 to $936,$248 to $268,$837 to $877,$724 to $762,$955 to $966,$331 to $340,$293 to $347,$89 to $93,$24 to $31
4,owensboro,Kentucky,42301,44883,$301 to $404,$775 to $936,$248 to $268,$837 to $877,$724 to $762,$955 to $966,$331 to $340,$293 to $347,$89 to $93,$24 to $31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,westminster,Colorado,80031,37254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,wheeling,West Virginia,26036,437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,wichita falls,Texas,76308,20305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,wilmington,North Carolina,28412,42758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Develop the scraper

In [2]:
url = 'https://repairpal.com/estimator/location'
user_agent = ""

This version works, but it's too long

In [ ]:
for index, zip_code in enumerate(top_zip_codes.zip[:1]):
    
    options = webdriver.ChromeOptions()
    options.add_argument(f'--user-agent={user_agent}')

    driver = uc.Chrome(use_subprocess=True, options=options) 
    driver.get(url)
    time.sleep(5)
    
    enter_zip = driver.find_element(by=By.ID, value='zipCode')
    enter_zip.send_keys(zip_code)
    time.sleep(5)
        
    enter_button = driver.find_element(by=By.XPATH,
                                     value='/html/body/div[1]/div/div/div/div[2]/div/div/div/div[1]/div/div/div[2]/div/div[2]/button')
    enter_button.click()
    time.sleep(10)
    dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span')
    dropdown.click()

    # Wait for the options to be visible 
    wait = WebDriverWait(driver, 10)
    options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

    # Loop through the options and select the one with text 'Honda'
    for option in options:
        if option.text == 'Honda':
            option.click()
            break
            
    time.sleep(5)        
    dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span')
    dropdown.click()

    # Wait for the options to be visible 
    wait = WebDriverWait(driver, 10)
    options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

    # Loop through the options and select the one with text '2021'
    for option in options:
        if option.text == '2021':
            option.click()
            break
            
    time.sleep(5)
    dropdown = driver.find_element(By.XPATH, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span')
    dropdown.click()

    # Wait for the options to be visible 
    wait = WebDriverWait(driver, 10)
    options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

    # Loop through the options and select the one with text 'Accord'
    for option in options:
        if option.text == 'Accord':
            option.click()
            break
            
    time.sleep(5)
    
    continue_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button')
    continue_button.click()
    
    time.sleep(5)
    
    # Repair 1: Brake Pad Replacement
    select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[1]/div[2]/div/button')
    select_button.click()
    time.sleep(5)
    
    input_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[2]/div/div[1]/label/input')
    input_button.click()
    time.sleep(5)
    

    submodel_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[3]/div/div[10]/label/input')
    submodel_button.click()
    time.sleep(5)
    

    submit_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button')
    submit_button.click()
    time.sleep(10)
    

    soup  = BeautifulSoup(driver.page_source)

    repair_price = soup.find_all("div", 'estimate-total-range')[0].text
    time.sleep(5)
    
    driver.quit()

Use functions to make code shorter

In [3]:
# Define a function for waiting until an element is clickable
def wait_for_clickable(driver, locator, timeout=10):
    return WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))

In [4]:
def select_option(driver, xpath, option_text):
    dropdown = wait_for_clickable(driver, (By.XPATH, xpath))
    dropdown.click()

    # Wait for the options to be visible 
    wait = WebDriverWait(driver, 10)
    options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

    # Find the option with the desired text
    option_to_select = None
    for option in options:
        if option.text == option_text:
            option_to_select = option
            break

    # Click the found option
    if option_to_select:
        option_to_select.click()
    else:
        print(f"Option '{option_text}' not found.")
    time.sleep(3)

## Scraper

In [5]:
to_scrape = pd.read_excel('scraped0612.xlsx', index_col=0)

In [6]:
to_scrape['zip'] = to_scrape['zip'].astype(str)
to_scrape['zip'] = to_scrape['zip'].apply(lambda x: x.zfill(5))

In [7]:
to_scrape[290:]

,City,State,zip,population,Brake Pad Replacement,Water Pump Replacement,Spark Plug Replacement,Alternator Replacement,Starter Replacement,Door Mirror Replacement,Battery Replacement,AC Recharge and Leak Inspection,"7,500 mile service",Tire Balancing
290,killeen,Texas,76549,56199,$388 to $515,"$1,045 to $1,281",$291 to $324,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,knoxville,Tennessee,37915,5509,$489 to $591,NaN,$321 to $361,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,lafayette,Louisiana,70508,40343,$342 to $456,"$902 to $1,098",$268 to $294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,lakeland,Florida,33810,52037,$478 to $576,NaN,$315 to $354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,lakewood,New Jersey,08701,135256,$493 to $596,"$1,243 to $1,533",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,westminster,Colorado,80031,37254,$522 to $632,"$1,331 to $1,645",$337 to $382,"$1,016 to $1,105",NaN,NaN,NaN,NaN,NaN,NaN
416,wheeling,West Virginia,26036,437,$301 to $404,NaN,$248 to $268,$837 to $877,NaN,NaN,NaN,NaN,NaN,NaN
417,wichita falls,Texas,76308,20305,$301 to $404,NaN,$248 to $268,$837 to $877,NaN,NaN,NaN,NaN,NaN,NaN
418,wilmington,North Carolina,28412,42758,$317 to $424,NaN,$256 to $278,$853 to $898,NaN,NaN,NaN,NaN,NaN,NaN


## Repair 1: Brake Pad Replacement

In [13]:
for index, row in to_scrape.iloc[408:].iterrows():
    zip_code = row['zip']
    try:
        options = webdriver.ChromeOptions()
        options.add_argument(f'--user-agent={user_agent}')
        options.add_argument('--headless')

        driver = uc.Chrome(use_subprocess=True, options=options)
        driver.get(url)

        enter_zip = driver.find_element(By.ID, 'zipCode')
        enter_zip.send_keys(zip_code)
        time.sleep(1)

        enter_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div/div/div/div/div[2]/div/div/div/div/div/div[2]/button'))
        enter_button.click()
        time.sleep(3)

        # Select options in dropdowns
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span', 'Honda')
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span', '2021')
        select_option(driver, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span', 'Accord')

        continue_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button'))
        continue_button.click()
        time.sleep(3)

        # Repair 1: Brake Pad Replacement
        select_button = wait_for_clickable(driver, (By.XPATH, '//*[@id="react-tabs-1"]/div/div[1]/div[2]/div/button'))
        select_button.click()
        time.sleep(1)

        input_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[2]/div/div[1]/label/input'))
        input_button.click()    

        try:

            submodel_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[3]/div/div[10]/label/input'))
            submodel_button.click()

            submit_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button'))
            submit_button.click()

        except:

            submit_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button'))
            submit_button.click()


        time.sleep(10)
        soup  = BeautifulSoup(driver.page_source)

        repair_price = soup.find_all("div", 'estimate-total-range')[0].text

        to_scrape.at[index, 'Brake Pad Replacement'] = repair_price
        to_scrape.to_excel('scraped0612.xlsx')

        driver.close()
        driver.quit()

        print(f'Row {index} is done')

    except: 
        print(f'Error {index}')
        
        try:
            driver.close()
            driver.quit()
        except:
            continue


Row 408 is done
Error 409
Row 410 is done
Row 411 is done
Error 412
Row 413 is done
Row 414 is done
Row 415 is done
Row 416 is done
Row 417 is done
Row 418 is done
Error 419


## Repair 2: Water Pump Replacement

In [12]:
for index, row in to_scrape.iloc[406:].iterrows():
    zip_code = row['zip']
    try:
        options = webdriver.ChromeOptions()
        options.add_argument(f'--user-agent={user_agent}')
        options.add_argument('--headless')

        driver = uc.Chrome(use_subprocess=True, options=options)
        driver.get(url)
        enter_zip = driver.find_element(By.ID, 'zipCode')
        enter_zip.send_keys(zip_code)
        time.sleep(1)

        enter_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div/div/div/div/div[2]/div/div/div/div/div/div[2]/button'))
        enter_button.click()
        time.sleep(3)

        # Select options in dropdowns
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span', 'Honda')
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span', '2021')
        select_option(driver, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span', 'Accord')

        continue_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button'))
        continue_button.click()
        time.sleep(5)

        # Repair 2: Water Pump Replacement
        select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[2]/div[2]/div/button')
        select_button.click()
        time.sleep(1)
            
        try:

            input_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[2]/div/div[1]/label/input'))
            input_button.click()
            time.sleep(1)

            submit_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button'))
            submit_button.click()
                
        except:
            time.sleep(0.01)

        time.sleep(8)
        soup  = BeautifulSoup(driver.page_source)

        repair_price = soup.find_all("div", 'estimate-total-range')[0].text

        to_scrape.at[index, 'Water Pump Replacement'] = repair_price
        to_scrape.to_excel('scraped0612.xlsx')

        driver.close()
        driver.quit()

        print(f'Row {index} is done')

    except: 
        print(f'Error {index}')
        
        try:
            driver.close()
            driver.quit()
        except:
            continue


Row 406 is done
Error 407
Error 408
Error 409
Error 410
Row 411 is done
Row 412 is done
Row 413 is done
Error 414
Row 415 is done
Error 416
Error 417
Error 418
Row 419 is done


## Repair 3: Spark Plug Replacement

In [9]:
for index, row in to_scrape.iloc[396:].iterrows():
    zip_code = row['zip']
    try:
        options = webdriver.ChromeOptions()
        options.add_argument(f'--user-agent={user_agent}')
        options.add_argument('--headless')

        driver = uc.Chrome(use_subprocess=True, options=options)
        driver.get(url)
        enter_zip = driver.find_element(By.ID, 'zipCode')
        enter_zip.send_keys(zip_code)
        time.sleep(1)

        enter_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div/div/div/div/div[2]/div/div/div/div/div/div[2]/button'))
        enter_button.click()
        time.sleep(3)

        # Select options in dropdowns
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span', 'Honda')
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span', '2021')
        select_option(driver, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span', 'Accord')

        continue_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button'))
        continue_button.click()
        time.sleep(5)

        # Repair 3: Spark Plug Replacement
        select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[4]/div[2]/div/button')
        select_button.click()
            
        input_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[2]/div/div[1]/label/input'))
        input_button.click()
        time.sleep(1)

        submit_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button'))
        submit_button.click()

        time.sleep(10)
        soup  = BeautifulSoup(driver.page_source)

        repair_price = soup.find_all("div", 'estimate-total-range')[0].text

        to_scrape.at[index, 'Spark Plug Replacement'] = repair_price
        to_scrape.to_excel('scraped0612.xlsx')

        driver.close()
        driver.quit()

        print(f'Row {index} is done')

    except: 
        print(f'Error {index}')
        
        try:
            driver.close()
            driver.quit()
        except:
            time.sleep(0.01)

Row 396 is done
Row 397 is done
Row 398 is done
Row 399 is done
Error 400
Row 401 is done
Error 402
Row 403 is done
Row 404 is done
Error 405
Error 406
Error 407
Row 408 is done
Row 409 is done
Row 410 is done
Row 411 is done
Error 412
Error 413
Error 414
Row 415 is done
Row 416 is done
Row 417 is done
Row 418 is done
Row 419 is done


## Repair 4: Alternator Replacement

In [8]:
for index, row in to_scrape.iloc[408:].iterrows():
    zip_code = row['zip']
    try:
        options = webdriver.ChromeOptions()
        options.add_argument(f'--user-agent={user_agent}')
        options.add_argument('--headless')

        driver = uc.Chrome(use_subprocess=True, options=options)
        driver.get(url)
        enter_zip = driver.find_element(By.ID, 'zipCode')
        enter_zip.send_keys(zip_code)
        time.sleep(1)

        enter_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div/div/div/div/div[2]/div/div/div/div/div/div[2]/button'))
        enter_button.click()
        time.sleep(3)

        # Select options in dropdowns
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span', 'Honda')
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span', '2021')
        select_option(driver, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span', 'Accord')

        continue_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button'))
        continue_button.click()
        time.sleep(5)

        # Repair 4: Alternator Replacement
        select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[5]/div[2]/div/button')
        select_button.click()

        time.sleep(10)
        soup  = BeautifulSoup(driver.page_source)

        repair_price = soup.find_all("div", 'estimate-total-range')[0].text

        to_scrape.at[index, 'Alternator Replacement'] = repair_price
        to_scrape.to_excel('scraped0612.xlsx')

        driver.close()
        driver.quit()

        print(f'Row {index} is done')

    except: 
        print(f'Error {index}')
        
        try:
            driver.close()
            driver.quit()
        except:
            time.sleep(0.01)

Error 408
Row 409 is done
Row 410 is done
Row 411 is done
Error 412
Row 413 is done
Row 414 is done
Row 415 is done
Row 416 is done
Row 417 is done
Row 418 is done
Row 419 is done


## Repair 5: Starter Replacement

In [11]:
for index, row in to_scrape.iloc[397:416].iterrows():
    zip_code = row['zip']
    try:
        options = webdriver.ChromeOptions()
        options.add_argument(f'--user-agent={user_agent}')
        options.add_argument('--headless')

        driver = uc.Chrome(use_subprocess=True, options=options)
        driver.get(url)
        enter_zip = driver.find_element(By.ID, 'zipCode')
        enter_zip.send_keys(zip_code)
        time.sleep(1)

        enter_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div/div/div/div/div[2]/div/div/div/div/div/div[2]/button'))
        enter_button.click()
        time.sleep(3)

        # Select options in dropdowns
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span', 'Honda')
        select_option(driver, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span', '2021')
        select_option(driver, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span', 'Accord')

        continue_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button'))
        continue_button.click()
        time.sleep(5)

        # Repair 5: Starter Replacement
        select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[6]/div[2]/div/button')
        select_button.click()
            
        input_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[2]/div/div[1]/label/input'))
        input_button.click()
        time.sleep(1)

        submit_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button'))
        submit_button.click()

        time.sleep(10)
        soup  = BeautifulSoup(driver.page_source)

        repair_price = soup.find_all("div", 'estimate-total-range')[0].text

        to_scrape.at[index, 'Starter Replacement'] = repair_price
        to_scrape.to_excel('scraped0612.xlsx')

        driver.close()
        driver.quit()

        print(f'Row {index} is done')

    except: 
        print(f'Error {index}')
        
        try:
            driver.close()
            driver.quit()
        except:
            time.sleep(0.01)

Row 397 is done
Row 398 is done
Row 399 is done
Row 400 is done
Row 401 is done
Row 402 is done
Error 403
Row 404 is done
Row 405 is done
Row 406 is done
Row 407 is done
Error 408
Row 409 is done
Row 410 is done
Row 411 is done
Error 412
Error 413
Row 414 is done
Row 415 is done


In [12]:
#top_zip_codes.to_excel('Brake Pad Replacement.xlsx')

# Code step by step

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument(f'--user-agent={user_agent}')
options.add_argument('--headless')

driver = uc.Chrome(use_subprocess=True, options=options) # webdriver.Chrome(options=options) 
#driver = webdriver.Chrome(options=options)
driver.get(url)

In [254]:
enter_zip = driver.find_element(by=By.ID, value='zipCode')
enter_zip.send_keys(zip_code)

In [255]:
#enter_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[1]/div/div/div[2]/div/div[2]/button')
enter_button = wait_for_clickable(driver, (By.XPATH, '/html/body/div/div/div/div/div[2]/div/div/div/div/div/div[2]/button'))
enter_button.click()

In [16]:
dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[2]/div/span')
dropdown.click()

# Wait for the options to be visible (adjust the timeout as needed)
wait = WebDriverWait(driver, 10)
options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

# Loop through the options and select the one with text 'Honda'
for option in options:
    if option.text == 'Honda':
        option.click()
        break

In [17]:
dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[2]/div[2]/div/div[2]/div/span')
dropdown.click()

# Wait for the options to be visible (adjust the timeout as needed)
wait = WebDriverWait(driver, 10)
options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

# Loop through the options and select the one with text 'Honda'
for option in options:
    if option.text == '2021':
        option.click()
        break

In [18]:
dropdown = driver.find_element(By.XPATH, '//*[@id="model-selector"]/div[2]/div/div[2]/div/span')
dropdown.click()

# Wait for the options to be visible (adjust the timeout as needed)
wait = WebDriverWait(driver, 10)
options = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//span[@class="select__option-text"]')))

# Loop through the options and select the one with text 'Honda'
for option in options:
    if option.text == 'Accord':
        option.click()
        break

In [19]:
continue_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[3]/div[4]/button')
continue_button.click()

In [ ]:
# Repair 1: Brake Pad Replacement
select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[1]/div[2]/div/button')
select_button.click()

In [ ]:
# Repair 2: Water Pump Replacement
select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[2]/div[2]/div/button')
select_button.click()

In [26]:
# Repair 3: Spark Plug Replacement
select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[4]/div[2]/div/button')
select_button.click()

In [19]:
# Repair 4: Alternator Replacement
select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[5]/div[2]/div/button')
select_button.click()

In [20]:
# Repair 5: Starter Replacement
select_button = driver.find_element(By.XPATH, '//*[@id="react-tabs-1"]/div/div[6]/div[2]/div/button')
select_button.click()

In [21]:
input_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[2]/div/div[1]/label/input')
input_button.click()

In [ ]:
submodel_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[6]/div[3]/div/div[10]/label/input')
submodel_button.click()

In [22]:
submit_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/div/div/div[2]/div[7]/button')
submit_button.click()

In [29]:
soup  = BeautifulSoup(driver.page_source)

In [30]:
repair_price = soup.find_all("div", 'estimate-total-range')[0].text

In [31]:
repair_price

'$249 to $269'

In [ ]:
driver.close()
driver.quit()

# Calculate the average price for each price range for each repair

In [125]:
df = pd.read_excel('all_zip_codes_0912.xlsx')

In [126]:
def convert_price_range_to_average(df, columns):
    for column in columns:
        df[column] = df[column].str.replace('$', '').str.replace(',', '')  # Remove dollar signs
        df[column] = df[column].str.split(' to ')  # Split the range into a list

        # Convert the range values to floats
        df[f'{column}_min'] = df[column].apply(lambda x: float(x[0]) if isinstance(x, list) else None)
        df[f'{column}_max'] = df[column].apply(lambda x: float(x[1]) if isinstance(x, list) else None)

        # Calculate the average price
        df[f'{column}_average'] = (df[f'{column}_min'] + df[f'{column}_max']) / 2

        # Drop unnecessary columns
        df = df.drop([column, f'{column}_min', f'{column}_max'], axis=1)

    return df

In [127]:
# Example usage
columns_to_convert = ['Brake Pad Replacement', 'Water Pump Replacement', 'Spark Plug Replacement',
                      'Alternator Replacement', 'Starter Replacement', 'Door Mirror Replacement',
                      'Battery Replacement', 'AC Recharge and Leak Inspection',
                      '7,500 mile service', 'Tire Balancing']

# Assuming your DataFrame is named 'df'
df = convert_price_range_to_average(df, columns_to_convert)

In [128]:
df.columns = ['City', 'State', 'zip', 'population', 'Brake Pad Replacement',
       'Water Pump Replacement', 'Spark Plug Replacement',
       'Alternator Replacement', 'Starter Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing']

In [129]:
df

,City,State,zip,population,Brake Pad Replacement,Water Pump Replacement,Spark Plug Replacement,Alternator Replacement,Starter Replacement,Door Mirror Replacement,Battery Replacement,AC Recharge and Leak Inspection,"7,500 mile service",Tire Balancing
0,missoula,Montana,59804,8193,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
1,cheyenne,Wyoming,82001,37528,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
2,charleston,West Virginia,25311,9487,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
3,south burlington,Vermont,5403,20166,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
4,owensboro,Kentucky,42301,44883,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,westminster,Colorado,80031,37254,577.0,1488.0,359.5,1060.5,935.0,1016.5,381.0,591.5,109.5,61.5
416,wheeling,West Virginia,26036,437,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
417,wichita falls,Texas,76308,20305,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5
418,wilmington,North Carolina,28412,42758,370.5,912.0,267.0,875.5,760.0,965.0,340.0,344.5,92.0,30.5


In [130]:
# Create a new column with the combined price for all repairs
df['Combined_Price'] = df[['Brake Pad Replacement', 'Water Pump Replacement', 'Spark Plug Replacement',
                           'Alternator Replacement', 'Starter Replacement', 'Door Mirror Replacement',
                           'Battery Replacement', 'AC Recharge and Leak Inspection',
                           '7,500 mile service', 'Tire Balancing']].sum(axis=1)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   City                             420 non-null    object 
 1   State                            420 non-null    object 
 2   zip                              420 non-null    int64  
 3   population                       420 non-null    int64  
 4   Brake Pad Replacement            420 non-null    float64
 5   Water Pump Replacement           420 non-null    float64
 6   Spark Plug Replacement           420 non-null    float64
 7   Alternator Replacement           420 non-null    float64
 8   Starter Replacement              420 non-null    float64
 9   Door Mirror Replacement          420 non-null    float64
 10  Battery Replacement              420 non-null    float64
 11  AC Recharge and Leak Inspection  420 non-null    float64
 12  7,500 mile service    

In [132]:
df

,City,State,zip,population,Brake Pad Replacement,Water Pump Replacement,Spark Plug Replacement,Alternator Replacement,Starter Replacement,Door Mirror Replacement,Battery Replacement,AC Recharge and Leak Inspection,"7,500 mile service",Tire Balancing,Combined_Price
0,missoula,Montana,59804,8193,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
1,cheyenne,Wyoming,82001,37528,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
2,charleston,West Virginia,25311,9487,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
3,south burlington,Vermont,5403,20166,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
4,owensboro,Kentucky,42301,44883,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,westminster,Colorado,80031,37254,577.0,1488.0,359.5,1060.5,935.0,1016.5,381.0,591.5,109.5,61.5,6580.0
416,wheeling,West Virginia,26036,437,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
417,wichita falls,Texas,76308,20305,352.5,855.5,258.0,857.0,743.0,960.5,335.5,320.0,91.0,27.5,4800.5
418,wilmington,North Carolina,28412,42758,370.5,912.0,267.0,875.5,760.0,965.0,340.0,344.5,92.0,30.5,4957.0


# Establish the national average price for each of the 10 repairs + for all repairs combined

Calculate the national average price per repair for all repairs in all cities combined + the national average price when combining all of the repairs into one price

In [133]:
brake_pad_replacement_avg = df['Brake Pad Replacement'].mean()
brake_pad_replacement_avg

497.8309523809524

In [134]:
water_pump_replacement_avg = df['Water Pump Replacement'].mean()
water_pump_replacement_avg

1269.3797619047618

In [135]:
spark_plug_replacement_avg = df['Spark Plug Replacement'].mean()
spark_plug_replacement_avg

324.7892857142857

In [136]:
alternator_replacement_avg = df['Alternator Replacement'].mean()
alternator_replacement_avg

991.1416666666667

In [137]:
starter_replacement_avg = df['Starter Replacement'].mean()
starter_replacement_avg

868.2916666666666

In [138]:
door_mirror_replacement_avg = df['Door Mirror Replacement'].mean()
door_mirror_replacement_avg

998.197619047619

In [139]:
battery_replacement_avg = df['Battery Replacement'].mean()
battery_replacement_avg

365.98333333333335

In [140]:
ac_recharge_avg = df['AC Recharge and Leak Inspection'].mean()
ac_recharge_avg

497.61309523809524

In [141]:
mile_service_7500_avg = df['7,500 mile service'].mean()
mile_service_7500_avg

103.1595238095238

In [142]:
tire_balancing_avg = df['Tire Balancing'].mean()
tire_balancing_avg

49.923809523809524

In [143]:
national_average = df['Combined_Price'].mean()
national_average

5966.310714285714

In [144]:
df[['Brake Pad Replacement', 'Water Pump Replacement', 'Spark Plug Replacement',
                       'Alternator Replacement', 'Starter Replacement', 'Door Mirror Replacement',
                       'Battery Replacement', 'AC Recharge and Leak Inspection',
                       '7,500 mile service', 'Tire Balancing']].mean()

Brake Pad Replacement               497.830952
Water Pump Replacement             1269.379762
Spark Plug Replacement              324.789286
Alternator Replacement              991.141667
Starter Replacement                 868.291667
Door Mirror Replacement             998.197619
Battery Replacement                 365.983333
AC Recharge and Leak Inspection     497.613095
7,500 mile service                  103.159524
Tire Balancing                       49.923810
dtype: float64

# Organise tabs and rankings

## US states

US states: the price premium paid for car repairs (total) in each U.S. state vs the national average

Columns: State, Price Premium for Car Repairs (%), Average Cost of All Repairs Combined (USD)

In [145]:
# Group by 'State' and calculate the Average Cost of All Repairs Combined for each state
us_states = df.groupby('State')['Combined_Price'].mean().reset_index()

# Rename the columns in the new DataFrame
us_states.columns = ['State', 'Average Cost of All Repairs Combined']

# Calculate the price premium (%)
us_states['Price Premium for Car Repairs (%)'] = ((us_states['Average Cost of All Repairs Combined'] - national_average) / national_average)

us_states

,State,Average Cost of All Repairs Combined,Price Premium for Car Repairs (%)
0,Alabama,5803.800000,-0.027238
1,Alaska,5019.000000,-0.158777
2,Arizona,6283.791667,0.053212
3,Arkansas,5302.400000,-0.111277
4,California,6692.126667,0.121652
5,Colorado,6172.458333,0.034552
6,Connecticut,5614.800000,-0.058916
7,Delaware,5646.500000,-0.053603
8,District of Columbia,6538.000000,0.095820
9,Florida,6409.500000,0.074282


## Top Bottom Cities

Top Bottom Cities: the 10 cities with the highest and 10 cities with lowest premiums paid for car repairs (total)

Columns: City, State, Price Premium for Car Repairs (%), Average Cost of All Repairs Combined (USD)

In [146]:
cities = df.copy()

cities.drop(columns=['zip', 'population', 'Brake Pad Replacement',
       'Water Pump Replacement', 'Spark Plug Replacement',
       'Alternator Replacement', 'Starter Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing', ], inplace=True)

# Rename the columns in the new DataFrame
cities.columns = ['City', 'State', 'Average Cost of All Repairs Combined']

# Calculate the price premium (%)
cities['Price Premium for Car Repairs (%)'] = ((cities['Average Cost of All Repairs Combined'] - national_average) / national_average)
cities['City'] = cities['City'].apply(lambda x: x.title())
cities

,City,State,Average Cost of All Repairs Combined,Price Premium for Car Repairs (%)
0,Missoula,Montana,4800.5,-0.195399
1,Cheyenne,Wyoming,4800.5,-0.195399
2,Charleston,West Virginia,4800.5,-0.195399
3,South Burlington,Vermont,4800.5,-0.195399
4,Owensboro,Kentucky,4800.5,-0.195399
...,...,...,...,...
415,Westminster,Colorado,6580.0,0.102859
416,Wheeling,West Virginia,4800.5,-0.195399
417,Wichita Falls,Texas,4800.5,-0.195399
418,Wilmington,North Carolina,4957.0,-0.169168


In [147]:
top_cities = cities.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).reset_index(drop=True).head(10)
top_cities

,City,State,Average Cost of All Repairs Combined,Price Premium for Car Repairs (%)
0,Concord,California,7274.0,0.219179
1,Richmond,California,7274.0,0.219179
2,San Mateo,California,7274.0,0.219179
3,Daly City,California,7274.0,0.219179
4,Antioch,California,7274.0,0.219179
5,Berkeley,California,7274.0,0.219179
6,San Francisco,California,7274.0,0.219179
7,Hayward,California,7274.0,0.219179
8,Fremont,California,7274.0,0.219179
9,Oakland,California,7274.0,0.219179


In [148]:
bottom_cities = cities.sort_values(by='Price Premium for Car Repairs (%)', ascending=True).reset_index(drop=True).head(10)
bottom_cities

,City,State,Average Cost of All Repairs Combined,Price Premium for Car Repairs (%)
0,Missoula,Montana,4800.5,-0.195399
1,Great Falls,Montana,4800.5,-0.195399
2,Columbia,Missouri,4800.5,-0.195399
3,Colchester,Vermont,4800.5,-0.195399
4,Burlington,Vermont,4800.5,-0.195399
5,Bangor,Maine,4800.5,-0.195399
6,Amarillo,Texas,4800.5,-0.195399
7,Wheeling,West Virginia,4800.5,-0.195399
8,Wichita Falls,Texas,4800.5,-0.195399
9,Huntington,West Virginia,4800.5,-0.195399


## Top 5 Cities per Repair

Top 5 Cities per Repair: the 5 cities with highest premiums paid (%) for car repairs (total) vs national average

Columns: City, State, Price Premium for Car Repairs (%), Average Cost of All Repairs Combined (USD)

In [149]:
# 1 Brake Pad Replacement
one = df.copy()

one.drop(columns=['zip', 'population',
       'Water Pump Replacement', 'Spark Plug Replacement',
       'Alternator Replacement', 'Starter Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
one.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
one['Price Premium for Car Repairs (%)'] = ((one['Replacement Price'] - brake_pad_replacement_avg) / brake_pad_replacement_avg)

one = one.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
one['Repair Type'] = 'Brake Pad Replacement'
one['City'] = one['City'].apply(lambda x: x.title())

one

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
218,Concord,California,657.0,7274.0,0.319725,Brake Pad Replacement
189,Berkeley,California,657.0,7274.0,0.319725,Brake Pad Replacement
374,San Mateo,California,657.0,7274.0,0.319725,Brake Pad Replacement
225,Daly City,California,657.0,7274.0,0.319725,Brake Pad Replacement
172,Antioch,California,657.0,7274.0,0.319725,Brake Pad Replacement


In [150]:
# 2 Water Pump Replacement
two = df.copy()

two.drop(columns=['zip', 'population',
       'Brake Pad Replacement', 'Spark Plug Replacement',
       'Alternator Replacement', 'Starter Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
two.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
two['Price Premium for Car Repairs (%)'] = ((two['Replacement Price'] - water_pump_replacement_avg) / water_pump_replacement_avg)

two = two.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
two['Repair Type'] = 'Water Pump Replacement'
two['City'] = two['City'].apply(lambda x: x.title())

two

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
218,Concord,California,1737.5,7274.0,0.368779,Water Pump Replacement
189,Berkeley,California,1737.5,7274.0,0.368779,Water Pump Replacement
374,San Mateo,California,1737.5,7274.0,0.368779,Water Pump Replacement
225,Daly City,California,1737.5,7274.0,0.368779,Water Pump Replacement
172,Antioch,California,1737.5,7274.0,0.368779,Water Pump Replacement


In [151]:
# 3 Spark Plug Replacement
three = df.copy()

three.drop(columns=['zip', 'population',
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Alternator Replacement', 'Starter Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
three.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
three['Price Premium for Car Repairs (%)'] = ((three['Replacement Price'] - spark_plug_replacement_avg) / spark_plug_replacement_avg)

three = three.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
three['Repair Type'] = 'Spark Plug Replacement'
three['City'] = three['City'].apply(lambda x: x.title())
three

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
218,Concord,California,400.0,7274.0,0.231568,Spark Plug Replacement
357,Richmond,California,400.0,7274.0,0.231568,Spark Plug Replacement
374,San Mateo,California,400.0,7274.0,0.231568,Spark Plug Replacement
225,Daly City,California,400.0,7274.0,0.231568,Spark Plug Replacement
172,Antioch,California,400.0,7274.0,0.231568,Spark Plug Replacement


In [152]:
# 4 Alternator Replacement
four = df.copy()

four.drop(columns=['zip', 'population',
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Starter Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
four.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
four['Price Premium for Car Repairs (%)'] = ((four['Replacement Price'] - alternator_replacement_avg) / alternator_replacement_avg)

four = four.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
four['Repair Type'] = 'Alternator Replacement'
four['City'] = four['City'].apply(lambda x: x.title())

four

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
189,Berkeley,California,1140.5,7274.0,0.150693,Alternator Replacement
329,Oakland,California,1140.5,7274.0,0.150693,Alternator Replacement
357,Richmond,California,1140.5,7274.0,0.150693,Alternator Replacement
251,Fremont,California,1140.5,7274.0,0.150693,Alternator Replacement
218,Concord,California,1140.5,7274.0,0.150693,Alternator Replacement


In [153]:
# 5 Starter Replacement
five = df.copy()

five.drop(columns=['zip', 'population', 
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Alternator Replacement',
       'Door Mirror Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
five.columns = ['City', 'State','Replacement Price','Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
five['Price Premium for Car Repairs (%)'] = ((five['Replacement Price'] - starter_replacement_avg) / starter_replacement_avg)

five = five.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
five['Repair Type'] = 'Starter Replacement'
five['City'] = five['City'].apply(lambda x: x.title())

five

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
251,Fremont,California,1010.5,7274.0,0.163779,Starter Replacement
270,Hayward,California,1010.5,7274.0,0.163779,Starter Replacement
329,Oakland,California,1010.5,7274.0,0.163779,Starter Replacement
374,San Mateo,California,1010.5,7274.0,0.163779,Starter Replacement
189,Berkeley,California,1010.5,7274.0,0.163779,Starter Replacement


In [154]:
# 6 Door Mirror Replacement
six = df.copy()

six.drop(columns=['zip', 'population', 
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Alternator Replacement',
       'Starter Replacement', 'Battery Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
six.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
six['Price Premium for Car Repairs (%)'] = ((six['Replacement Price'] - door_mirror_replacement_avg) / door_mirror_replacement_avg)

six = six.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
six['Repair Type'] = 'Door Mirror Replacement'
six['City'] = six['City'].apply(lambda x: x.title())
six

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
82,Kailua,Hawaii,1181.5,6676.0,0.183633,Door Mirror Replacement
80,Honolulu,Hawaii,1181.5,6676.0,0.183633,Door Mirror Replacement
81,Pearl City,Hawaii,1181.5,6676.0,0.183633,Door Mirror Replacement
225,Daly City,California,1039.0,7274.0,0.040876,Door Mirror Replacement
164,San Francisco,California,1039.0,7274.0,0.040876,Door Mirror Replacement


In [155]:
# 7 Battery Replacement
seven = df.copy()

seven.drop(columns=['zip', 'population', 
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Alternator Replacement',
       'Starter Replacement', 'Door Mirror Replacement',
       'AC Recharge and Leak Inspection', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
seven.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
seven['Price Premium for Car Repairs (%)'] = ((seven['Replacement Price'] - battery_replacement_avg) / battery_replacement_avg)

seven = seven.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
seven['Repair Type'] = 'Battery Replacement'
seven['City'] = seven['City'].apply(lambda x: x.title())

seven

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
82,Kailua,Hawaii,427.0,6676.0,0.166720,Battery Replacement
80,Honolulu,Hawaii,427.0,6676.0,0.166720,Battery Replacement
81,Pearl City,Hawaii,427.0,6676.0,0.166720,Battery Replacement
41,Hilo,Hawaii,403.5,5515.5,0.102509,Battery Replacement
189,Berkeley,California,399.0,7274.0,0.090214,Battery Replacement


In [156]:
# 8 AC Recharge and Leak Inspection
eight = df.copy()

eight.drop(columns=['zip', 'population', 
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Alternator Replacement',
       'Starter Replacement', 'Door Mirror Replacement',
       'Battery Replacement', '7,500 mile service',
       'Tire Balancing'], inplace=True)

# Rename the columns in the new DataFrame
eight.columns = ['City', 'State','Replacement Price',  'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
eight['Price Premium for Car Repairs (%)'] = ((eight['Replacement Price'] - ac_recharge_avg) / ac_recharge_avg)

eight = eight.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
eight['Repair Type'] = 'AC Recharge and Leak Inspection'
eight['City'] = eight['City'].apply(lambda x: x.title())

eight

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
218,Concord,California,698.5,7274.0,0.403701,AC Recharge and Leak Inspection
189,Berkeley,California,698.5,7274.0,0.403701,AC Recharge and Leak Inspection
374,San Mateo,California,698.5,7274.0,0.403701,AC Recharge and Leak Inspection
225,Daly City,California,698.5,7274.0,0.403701,AC Recharge and Leak Inspection
172,Antioch,California,698.5,7274.0,0.403701,AC Recharge and Leak Inspection


In [157]:
# 9 7,500 mile service
nine = df.copy()

nine.drop(columns=['zip', 'population', 
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Alternator Replacement',
       'Starter Replacement', 'Door Mirror Replacement',
       'Battery Replacement', 'AC Recharge and Leak Inspection',
       'Tire Balancing'], inplace=True)

nine.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
nine['Price Premium for Car Repairs (%)'] = ((nine['Replacement Price'] - mile_service_7500_avg) / mile_service_7500_avg)

nine = nine.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
nine['Repair Type'] = '7,500 mile service'
nine['City'] = nine['City'].apply(lambda x: x.title())

nine

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
81,Pearl City,Hawaii,118.5,6676.0,0.148706,"7,500 mile service"
80,Honolulu,Hawaii,118.5,6676.0,0.148706,"7,500 mile service"
82,Kailua,Hawaii,118.5,6676.0,0.148706,"7,500 mile service"
374,San Mateo,California,117.0,7274.0,0.134166,"7,500 mile service"
377,Santa Clara,California,117.0,7260.5,0.134166,"7,500 mile service"


In [158]:
# 10 Tire Balancing
ten = df.copy()

ten.drop(columns=['zip', 'population', 
       'Brake Pad Replacement', 'Water Pump Replacement',
       'Spark Plug Replacement', 'Alternator Replacement',
       'Starter Replacement', 'Door Mirror Replacement',
       'Battery Replacement', 'AC Recharge and Leak Inspection',
       '7,500 mile service'], inplace=True)

ten.columns = ['City', 'State', 'Replacement Price', 'Average Cost of All Repairs Combined (USD)']

# Calculate the price premium (%)
ten['Price Premium for Car Repairs (%)'] = ((ten['Replacement Price'] - tire_balancing_avg) / tire_balancing_avg)

ten = ten.sort_values(by='Price Premium for Car Repairs (%)', ascending=False).head(5)
ten['Repair Type'] = 'Tire Balancing'
ten['City'] = ten['City'].apply(lambda x: x.title())

ten

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
338,Pasadena,Texas,89.0,6878.5,0.782717,Tire Balancing
270,Hayward,California,75.0,7274.0,0.502289,Tire Balancing
225,Daly City,California,75.0,7274.0,0.502289,Tire Balancing
189,Berkeley,California,75.0,7274.0,0.502289,Tire Balancing
329,Oakland,California,75.0,7274.0,0.502289,Tire Balancing


In [159]:
top_5_cities = pd.concat([one, two, three, four, five, six, seven, eight, nine, ten])
top_5_cities

,City,State,Replacement Price,Average Cost of All Repairs Combined (USD),Price Premium for Car Repairs (%),Repair Type
218,Concord,California,657.0,7274.0,0.319725,Brake Pad Replacement
189,Berkeley,California,657.0,7274.0,0.319725,Brake Pad Replacement
374,San Mateo,California,657.0,7274.0,0.319725,Brake Pad Replacement
225,Daly City,California,657.0,7274.0,0.319725,Brake Pad Replacement
172,Antioch,California,657.0,7274.0,0.319725,Brake Pad Replacement
218,Concord,California,1737.5,7274.0,0.368779,Water Pump Replacement
189,Berkeley,California,1737.5,7274.0,0.368779,Water Pump Replacement
374,San Mateo,California,1737.5,7274.0,0.368779,Water Pump Replacement
225,Daly City,California,1737.5,7274.0,0.368779,Water Pump Replacement
172,Antioch,California,1737.5,7274.0,0.368779,Water Pump Replacement


## New York

1x Tab per Major City: New York, Los Angeles, Philadelphia, Miami  — showing the price premium paid repair for each city vs the national average

Columns: Repair Type, Price Premium (%), Average Cost of Repair In City (USD)

In [160]:
avarage = pd.DataFrame(df[['Brake Pad Replacement', 'Water Pump Replacement', 'Spark Plug Replacement',
                       'Alternator Replacement', 'Starter Replacement', 'Door Mirror Replacement',
                       'Battery Replacement', 'AC Recharge and Leak Inspection',
                       '7,500 mile service', 'Tire Balancing']].mean()).reset_index()
avarage.columns = ['Repair Type', 'average_cost']

In [161]:
avarage

,Repair Type,average_cost
0,Brake Pad Replacement,497.830952
1,Water Pump Replacement,1269.379762
2,Spark Plug Replacement,324.789286
3,Alternator Replacement,991.141667
4,Starter Replacement,868.291667
5,Door Mirror Replacement,998.197619
6,Battery Replacement,365.983333
7,AC Recharge and Leak Inspection,497.613095
8,"7,500 mile service",103.159524
9,Tire Balancing,49.923810


In [162]:
new_york = df[df['City']=='new york']

new_york.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)

# Melt the DataFrame
new_york = pd.melt(new_york, var_name='Repair Type', value_name='Average Cost of Repair In City (USD)') 

new_york['Price Premium (%)'] = ((new_york['Average Cost of Repair In City (USD)'] - avarage.average_cost) / avarage.average_cost)

new_york

/var/folders/9l/y7rkpr294hgfdlnpg_23jdz40000gn/T/ipykernel_1059/3421690122.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_york.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)


,Repair Type,Average Cost of Repair In City (USD),Price Premium (%)
0,Brake Pad Replacement,544.5,0.093745
1,Water Pump Replacement,1388.0,0.093447
2,Spark Plug Replacement,343.5,0.057609
3,Alternator Replacement,1028.0,0.037188
4,Starter Replacement,904.5,0.041701
5,Door Mirror Replacement,1007.5,0.009319
6,Battery Replacement,374.0,0.021904
7,AC Recharge and Leak Inspection,548.5,0.102262
8,"7,500 mile service",106.5,0.032382
9,Tire Balancing,56.0,0.121709


## Los Angeles

In [163]:
los_angeles = df[df['City']=='los angeles']

los_angeles.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)

# Melt the DataFrame
los_angeles = pd.melt(los_angeles, var_name='Repair Type', value_name='Average Cost of Repair In City (USD)') 

los_angeles['Price Premium (%)'] = ((los_angeles['Average Cost of Repair In City (USD)'] - avarage.average_cost) / avarage.average_cost)

los_angeles

/var/folders/9l/y7rkpr294hgfdlnpg_23jdz40000gn/T/ipykernel_1059/3896574412.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  los_angeles.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)


,Repair Type,Average Cost of Repair In City (USD),Price Premium (%)
0,Brake Pad Replacement,649.0,0.303655
1,Water Pump Replacement,1712.5,0.349084
2,Spark Plug Replacement,396.0,0.219252
3,Alternator Replacement,1132.5,0.142622
4,Starter Replacement,1003.0,0.155142
5,Door Mirror Replacement,1037.0,0.038872
6,Battery Replacement,397.0,0.084749
7,AC Recharge and Leak Inspection,687.5,0.381595
8,"7,500 mile service",116.0,0.124472
9,Tire Balancing,74.0,0.482259


## Philadelphia

In [164]:
philadelphia = df[df['City']=='philadelphia']

philadelphia.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)

# Melt the DataFrame
philadelphia = pd.melt(philadelphia, var_name='Repair Type', value_name='Average Cost of Repair In City (USD)') 

philadelphia['Price Premium (%)'] = ((philadelphia['Average Cost of Repair In City (USD)'] - avarage.average_cost) / avarage.average_cost)

philadelphia

/var/folders/9l/y7rkpr294hgfdlnpg_23jdz40000gn/T/ipykernel_1059/1988678360.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  philadelphia.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)


,Repair Type,Average Cost of Repair In City (USD),Price Premium (%)
0,Brake Pad Replacement,528.5,0.061605
1,Water Pump Replacement,1338.0,0.054058
2,Spark Plug Replacement,335.5,0.032977
3,Alternator Replacement,1012.0,0.021045
4,Starter Replacement,889.5,0.024425
5,Door Mirror Replacement,1003.0,0.004811
6,Battery Replacement,370.5,0.012341
7,AC Recharge and Leak Inspection,527.0,0.059056
8,"7,500 mile service",105.0,0.017841
9,Tire Balancing,53.5,0.071633


## Miami

In [165]:
miami = df[df['City']=='miami']

miami.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)

# Melt the DataFrame
miami = pd.melt(miami, var_name='Repair Type', value_name='Average Cost of Repair In City (USD)') 

miami['Price Premium (%)'] = ((miami['Average Cost of Repair In City (USD)'] - avarage.average_cost) / avarage.average_cost)

miami

/var/folders/9l/y7rkpr294hgfdlnpg_23jdz40000gn/T/ipykernel_1059/1918527888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miami.drop(columns=['zip', 'population', 'City', 'State', 'Combined_Price'], inplace=True)


,Repair Type,Average Cost of Repair In City (USD),Price Premium (%)
0,Brake Pad Replacement,601.0,0.207237
1,Water Pump Replacement,1563.0,0.231310
2,Spark Plug Replacement,372.0,0.145358
3,Alternator Replacement,1084.5,0.094193
4,Starter Replacement,958.0,0.103316
5,Door Mirror Replacement,1023.5,0.025348
6,Battery Replacement,386.5,0.056059
7,AC Recharge and Leak Inspection,623.5,0.252981
8,"7,500 mile service",111.5,0.080850
9,Tire Balancing,66.0,0.322014


# Create an Excel file

In [166]:
# Create an Excel writer object

writer = pd.ExcelWriter('The U.S. States and Cities that Pay the Biggest Premium for Car Repairs.xlsx', engine='openpyxl')

# Save each DataFrame to a separate sheet in the Excel file

us_states.to_excel(writer, sheet_name='US States', index=False)
top_cities.to_excel(writer, sheet_name='Top Cities', index=False)
bottom_cities.to_excel(writer, sheet_name='Bottom Cities', index=False)
top_5_cities.to_excel(writer, sheet_name='Top 5 Cities per Repair', index=False)
new_york.to_excel(writer, sheet_name='New York', index=False)
los_angeles.to_excel(writer, sheet_name='Los Angeles', index=False)
philadelphia.to_excel(writer, sheet_name='Philadelphia', index=False)
miami.to_excel(writer, sheet_name='Miami', index=False)
df.to_excel(writer, sheet_name='Raw Data', index=False)

# Save the Excel file
writer.close()